In [2]:
import numpy as np
import mylibrary.nnlib as tnn
import matplotlib.pyplot as plt
import copy

from mpl_toolkits.mplot3d import Axes3D
import matplotlib

import torch
import torch.nn as nn

import mylibrary.datasets as datasets

In [3]:
mnist = datasets.MNIST()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

In [4]:
train_label = tnn.Logits.index_to_logit(train_label_)
train_size = len(train_label_)

In [8]:
xx = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
yy = torch.LongTensor(train_label_)

In [6]:
net = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 10),
#     nn.Softmax(dim=1)
)

In [7]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.003)
criterion = nn.CrossEntropyLoss()

In [20]:
for epoch in range(100):
    yout = net(xx)

    loss = criterion(yout, yy)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    error = float(loss)
    print(epoch, 'Error = ', error)
    
    with torch.no_grad():
        yout = net(train_data)
        out = torch.argmax(yout, axis=1)
#         print(out.shape)
#         print(np.array(train_label_).shape)
        acc = (out.data.numpy() == np.array(train_label_)).astype(np.float).mean()
        print("Accuracy: ", acc)

0 Error =  0.35279321670532227
Accuracy:  0.9032166666666667
1 Error =  0.34422287344932556
Accuracy:  0.9061166666666667
2 Error =  0.3359905779361725
Accuracy:  0.9057333333333333
3 Error =  0.33292174339294434
Accuracy:  0.9085333333333333
4 Error =  0.31938502192497253
Accuracy:  0.90815
5 Error =  0.3179899752140045
Accuracy:  0.9106
6 Error =  0.3103329539299011
Accuracy:  0.9138
7 Error =  0.3011729121208191
Accuracy:  0.9153333333333333
8 Error =  0.2993481159210205
Accuracy:  0.9182833333333333
9 Error =  0.29042762517929077
Accuracy:  0.9189
10 Error =  0.2861914038658142
Accuracy:  0.9194833333333333
11 Error =  0.281129390001297
Accuracy:  0.9214166666666667
12 Error =  0.2754765748977661
Accuracy:  0.9233
13 Error =  0.27143123745918274
Accuracy:  0.9249
14 Error =  0.26632726192474365
Accuracy:  0.92625
15 Error =  0.26193252205848694
Accuracy:  0.9268666666666666
16 Error =  0.25749829411506653
Accuracy:  0.9268333333333333
17 Error =  0.25309884548187256
Accuracy:  0.92

In [24]:
list(net._modules.items())

[('0', Linear(in_features=784, out_features=256, bias=True)),
 ('1', ReLU()),
 ('2', Linear(in_features=256, out_features=128, bias=True)),
 ('3', ReLU()),
 ('4', Linear(in_features=128, out_features=64, bias=True)),
 ('5', ReLU()),
 ('6', Linear(in_features=64, out_features=10, bias=True))]

## Oracle Pruning

In [25]:
class Oracle():
    
    def __init__(self, net):
        self.net = net
        self.activations = {}
        self.gradients = {}
        self.forward_hook = {}
        self.backward_hook = {}
        self.keys = []
        for name, module in list(self.net._modules.items()):
            if isinstance(module, torch.nn.Linear):
                hook = module.register_backward_hook(self.capture_gradients)
                self.backward_hook[module] = hook
                hook = module.register_forward_hook(self.capture_inputs)
                self.forward_hook[module] = hook
                
                self.activations[module] = None
                self.gradients[module] = None
                self.keys.append(module)
        pass
            
        
    def capture_inputs(self, module, inp, out):
#         print(out.shape)
        self.activations[module] = out.data
        
    def capture_gradients(self, module, gradi, grado):
#         print("Grad")
#         print(gradi[-1])
#         for gi in gradi:
#             print(gi.shape)
#         print(grado)
#         print(grado[0].shape)
        self.gradients[module] = grado[0]
        
    def gather_inputs_gradients(self, x, t):
        self.net.zero_grad()
        y = self.net(x)
        
        error = criterion(y, t)
        error.backward()
        
        for module in self.keys:
            hook = self.forward_hook[module]
            hook.remove()
            hook = self.backward_hook[module]
            hook.remove()
        return
    
    def compute_significance(self, x, t):
        self.gather_inputs_gradients(x, t)
        
        ## compute importance score
        importance = []
        for module in self.keys:
            z = self.activations[module] * self.gradients[module]
            z = z.mean(dim=0).abs()
            importance.append(z)
            
        return importance

In [26]:
oracle = Oracle(net)
oracle.compute_significance(xx, yy)

[tensor([8.5159e-10, 2.2340e-09, 0.0000e+00, 1.4452e-09, 1.8545e-09, 1.3532e-10,
         1.0369e-09, 9.3642e-10, 1.3956e-09, 1.4765e-11, 3.5169e-09, 3.3023e-09,
         8.0959e-10, 1.7632e-09, 2.5364e-09, 7.2729e-10, 2.4542e-09, 1.8144e-09,
         2.5195e-09, 3.0664e-10, 2.0408e-09, 1.3529e-09, 7.3862e-10, 2.1734e-10,
         2.2921e-09, 4.5361e-11, 5.3660e-09, 1.4120e-09, 1.5193e-09, 1.1403e-09,
         3.9473e-09, 1.8232e-09, 1.4800e-09, 0.0000e+00, 5.9791e-11, 1.4199e-09,
         0.0000e+00, 5.8153e-10, 0.0000e+00, 1.9371e-09, 0.0000e+00, 8.9673e-10,
         4.2191e-10, 0.0000e+00, 1.2422e-09, 1.8172e-09, 0.0000e+00, 1.5679e-09,
         2.0490e-09, 2.2882e-09, 1.8317e-09, 6.2868e-09, 1.4803e-09, 4.7809e-09,
         2.0764e-09, 0.0000e+00, 1.3784e-09, 1.1671e-09, 8.6811e-10, 4.6927e-09,
         5.1318e-10, 2.4203e-09, 0.0000e+00, 9.0236e-10, 1.7595e-09, 2.2716e-09,
         3.7795e-10, 8.4219e-09, 1.2402e-09, 1.4659e-08, 7.2019e-10, 1.0749e-09,
         1.1212e-09, 0.0000e

In [141]:
class Oracle_Modified():
    
    def __init__(self, net, mode="oracle"):
        self.net = net
        self.mode = mode
        
        self.activations = {}
        self.gradients = {}
        self.forward_hook = {}
        self.backward_hook = {}
        self.keys = []
        for name, module in list(self.net._modules.items()):
            if isinstance(module, torch.nn.Linear):
                hook = module.register_backward_hook(self.capture_gradients)
                self.backward_hook[module] = hook
                hook = module.register_forward_hook(self.capture_inputs)
                self.forward_hook[module] = hook
                
                self.activations[module] = None
                self.gradients[module] = None
                self.keys.append(module)
        
#         self.importance = None
        pass
            
        
    def capture_inputs(self, module, inp, out):
        self.activations[module] = out.data
        
    def capture_gradients(self, module, gradi, grado):
        self.gradients[module] = grado[0]
        
    def gather_inputs_gradients(self, x, t):
        self.net.zero_grad()
        y = self.net(x)
        
        error = criterion(y,t)
        error.backward()
        
        for module in self.keys:
            hook = self.forward_hook[module]
            hook.remove()
            hook = self.backward_hook[module]
            hook.remove()
        return
    
    def compute_significance(self, x, t, mode=None, normalize=False):
        self.gather_inputs_gradients(x, t)
        
        if mode is None:
            mode = self.mode
        
        ## compute importance score
        importance = []
        if mode == "oracle":
            for module in self.keys:
                z = self.activations[module] * self.gradients[module]
                z = z.sum(dim=0).abs()
                importance.append(z)

        elif mode == "oracle_absolute":
            for module in self.keys:
                z = (self.activations[module] * self.gradients[module])
#                 z = z.abs().sum(dim=0)
                z = z.pow(2).sum(dim=0)
                importance.append(z)
        
        elif mode == "oracle_normalized":
            scaler = torch.norm(self.gradients[self.keys[-1]], dim=1, keepdim=True) + 1e-5
#             print(scaler.shape)
            for module in self.keys:
                z = (self.activations[module] * self.gradients[module])/scaler
                z = z.sum(dim=0).abs()
                importance.append(z)
                                
        elif mode == "oracle_abs_norm":
            scaler = torch.norm(self.gradients[self.keys[-1]], p=2, dim=1, keepdim=True) + 1e-5
#             print(scaler)
            for module in self.keys:
                z = (self.activations[module] * self.gradients[module])/scaler
                z = z.abs().sum(dim=0)
#                 z = z.pow(2).sum(dim=0)
                importance.append(z)               
        
        importance = importance[:-1]
        if normalize:
            sums = 0
            count = 0
            for imp in importance:
                sums += imp.sum()
                count += len(imp)
#             print(sums, count)
            divider = sums/count ## total importance is number of neurons
            for i in range(len(importance)):
                importance[i] = importance[i]/divider
            
        return importance

In [142]:
oracle_mod = Oracle_Modified(net)

In [143]:
modes = ["oracle", "oracle_absolute", "oracle_normalized", "oracle_abs_norm"]

In [144]:
oracle_mod.compute_significance(xx, yy, mode=modes[0], normalize=True)

[tensor([2.7180e-01, 7.1302e-01, 0.0000e+00, 4.6125e-01, 5.9188e-01, 4.3188e-02,
         3.3094e-01, 2.9887e-01, 4.4542e-01, 4.7126e-03, 1.1225e+00, 1.0540e+00,
         2.5839e-01, 5.6276e-01, 8.0953e-01, 2.3213e-01, 7.8328e-01, 5.7911e-01,
         8.0413e-01, 9.7869e-02, 6.5135e-01, 4.3181e-01, 2.3574e-01, 6.9368e-02,
         7.3158e-01, 1.4478e-02, 1.7127e+00, 4.5066e-01, 4.8489e-01, 3.6393e-01,
         1.2598e+00, 5.8190e-01, 4.7238e-01, 0.0000e+00, 1.9083e-02, 4.5319e-01,
         0.0000e+00, 1.8560e-01, 0.0000e+00, 6.1826e-01, 0.0000e+00, 2.8621e-01,
         1.3466e-01, 0.0000e+00, 3.9646e-01, 5.7998e-01, 0.0000e+00, 5.0043e-01,
         6.5398e-01, 7.3030e-01, 5.8462e-01, 2.0065e+00, 4.7246e-01, 1.5259e+00,
         6.6273e-01, 0.0000e+00, 4.3994e-01, 3.7250e-01, 2.7707e-01, 1.4977e+00,
         1.6379e-01, 7.7247e-01, 0.0000e+00, 2.8800e-01, 5.6158e-01, 7.2502e-01,
         1.2063e-01, 2.6880e+00, 3.9583e-01, 4.6785e+00, 2.2986e-01, 3.4308e-01,
         3.5786e-01, 0.0000e

In [145]:
oracle_mod.compute_significance(xx, yy, mode=modes[1], normalize=True)

[tensor([ 0.4430,  0.0295,  0.0000,  0.1659,  0.2032,  0.1729,  0.1887,  0.2304,
          0.0856,  0.1015,  0.2566,  0.3412,  0.1336,  0.0799,  0.3109,  0.3287,
          0.0927,  0.1783,  0.3302,  0.1829,  0.1044,  0.0370,  0.1718,  0.1816,
          0.0439,  0.0691,  0.1097,  0.0572,  0.1989,  0.2707,  0.1961,  0.1230,
          0.1407,  0.0000,  0.0468,  0.2188,  0.0000,  0.1459,  0.0000,  0.2626,
          0.0000,  0.0808,  0.2883,  0.0000,  0.0339,  0.2565,  0.0000,  0.2341,
          0.1036,  0.1811,  0.2531,  0.0888,  0.2588,  0.4036,  0.2096,  0.0000,
          0.3346,  0.2301,  0.2726,  0.4533,  5.6797,  0.2210,  0.0000,  0.1348,
          0.0679,  0.3212,  0.2382,  0.7364,  0.0275,  7.7658,  0.2186,  0.1298,
          0.3976,  0.0000,  0.5514,  0.0588,  0.1190,  0.2136,  0.1831,  0.0000,
          0.1302,  6.7365,  0.0000,  0.0000,  0.8187,  0.1947,  0.1121,  0.1243,
          0.1052,  1.1182,  0.3266,  0.3680,  0.1048,  0.1245,  0.1369,  0.0670,
          0.8273,  0.0900,  

In [146]:
oracle_mod.compute_significance(xx, yy, mode=modes[2], normalize=False)

[tensor([18.7008, 17.2124,  0.0000, 18.8252, 17.7527,  9.8356, 19.3307,  8.0831,
         19.3016, 10.0615, 26.9800, 27.1140, 20.8047, 11.8404, 18.7308, 28.9580,
         17.6353, 15.3055, 23.9021, 13.0712, 23.2089,  5.5523, 22.3621, 16.1522,
          6.1595,  4.3828, 30.8578, 17.9765,  3.8927,  8.2727, 32.7008, 23.6238,
         22.8824,  0.0000,  5.8859,  3.9255,  0.0000, 18.5689,  0.0000, 21.9841,
          0.0000,  8.2741, 23.8558,  0.0000,  9.9601,  8.1086,  0.0000,  1.7687,
         10.4644, 12.8090, 27.3795, 27.8015,  1.1915, 26.8675, 13.7592,  0.0000,
         21.3335,  1.5240, 26.4493, 27.1177, 10.6793, 20.6754,  0.0000, 13.3105,
          7.5875, 17.6572,  2.7125, 58.2491, 14.1572, 61.3366, 11.3919,  8.5474,
         18.3741,  0.0000, 27.4239,  0.6300, 10.0370,  4.5102,  7.7792,  0.0000,
          7.7157, 89.8425,  0.0000,  0.0000, 38.8277, 12.7981, 24.9589, 14.5716,
         12.7680, 43.5608, 30.8543,  6.3961, 14.7989, 10.3001, 14.2710,  3.8125,
         21.3587, 12.2650,  

In [147]:
oracle_mod.compute_significance(xx, yy, mode=modes[3], normalize=True)

[tensor([0.9568, 0.1873, 0.0000, 0.4858, 0.4294, 0.5639, 0.6374, 0.6543, 0.3412,
         0.3774, 0.6058, 0.9263, 0.4005, 0.3072, 0.7555, 0.8900, 0.4419, 0.4136,
         0.8522, 0.6309, 0.4823, 0.2580, 0.4870, 0.5977, 0.2362, 0.3728, 0.2536,
         0.2668, 0.6639, 0.7721, 0.5009, 0.4780, 0.5293, 0.0000, 0.2381, 0.6056,
         0.0000, 0.4263, 0.0000, 0.5496, 0.0000, 0.4056, 0.6556, 0.0000, 0.1994,
         0.7624, 0.0000, 0.4966, 0.4863, 0.6035, 0.6720, 0.3221, 0.7453, 0.9458,
         0.4804, 0.0000, 0.7611, 0.7352, 0.6815, 0.6337, 1.9885, 0.5199, 0.0000,
         0.4648, 0.2490, 0.8173, 0.6866, 0.8768, 0.1923, 2.6001, 0.4840, 0.5367,
         0.8049, 0.0000, 0.9763, 0.3590, 0.4352, 0.6047, 0.5474, 0.0000, 0.4659,
         2.4419, 0.0000, 0.0000, 0.8745, 0.6818, 0.3374, 0.2981, 0.4229, 1.1137,
         0.5367, 0.8340, 0.2817, 0.5369, 0.4976, 0.3909, 1.0648, 0.3270, 0.7534,
         0.1697, 2.4210, 0.5860, 0.3401, 0.6916, 0.7055, 0.4560, 0.8908, 0.2610,
         0.0000, 0.3835, 0.4

## Get pruning mask

In [148]:
def get_pruning_mask(importance, output_dim, num_prune=1):
    layer_dims = []
    for imp in importance:
        layer_dims.append(len(imp))
    
    imps = torch.ones(len(imp), max(layer_dims))*sum(layer_dims)*10
    imps_shape = imps.shape
    for i, imp in enumerate(importance):
        imps[i, :len(imp)] = imp
        
#     print(imps)
    imps = imps.reshape(-1)
    indices = torch.argsort(imps)
#     print(indices)
    imps[indices[:num_prune]] = -1.
    imps = imps.reshape(imps_shape)
    
    mask = (imps>=0).type(torch.float)
    masks = []
    for i, imp in enumerate(importance):
        masks.append(mask[i, :len(imp)])
    return masks

In [149]:
# importance = oracle_mod.compute_significance(xx, yy, mode=modes[3], normalize=True)
# importance

In [150]:
# get_pruning_mask(importance, 1, num_prune=7)

In [151]:
num = 10

In [152]:
importance = oracle_mod.compute_significance(xx, yy, mode=modes[0], normalize=True)
get_pruning_mask(importance, 1, num_prune=num)

[tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

In [153]:
importance = oracle_mod.compute_significance(xx, yy, mode=modes[1], normalize=True)
get_pruning_mask(importance, 1, num_prune=num)

[tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

In [154]:
importance = oracle_mod.compute_significance(xx, yy, mode=modes[2], normalize=True)
get_pruning_mask(importance, 1, num_prune=num)

[tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
         0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

In [155]:
importance = oracle_mod.compute_significance(xx, yy, mode=modes[3], normalize=True)
get_pruning_mask(importance, 1, num_prune=num)

[tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

## Define pruning function

In [156]:
class Pruner():
    
    def __init__(self, net, prune_mask=None):
        self.net = net
        self.keys = []
        self.prune_mask = {}
        self.forward_hook = {}
        
        self.activations = []
        
        for name, module in list(self.net._modules.items()):
            if isinstance(module, torch.nn.Linear):
                self.keys.append(module)

        if prune_mask is not None:
            self.add_prune_mask(prune_mask)
        self.remove_hook()
        
    def add_prune_mask(self, prune_mask):
        for module, pm in zip(self.keys[:-1], prune_mask):
            self.prune_mask[module] = pm.type(torch.float)
        self.prune_mask[self.keys[-1]] = torch.ones(self.keys[-1].out_features, dtype=torch.float)
            
        
    def prune_neurons(self, module, inp, out):
        mask = self.prune_mask[module]
        output = out*mask
        
        self.activations.append(output)
        return output
        
    def forward(self, x, prune_mask=None):
        if prune_mask:
            self.add_prune_mask(prune_mask)
            if len(self.forward_hook) == 0:
                self.add_hook()
        
        y = self.net(x)
        self.remove_hook()
        return y
        
        
    def add_hook(self):
        if len(self.forward_hook) > 0:
            self.remove_hook()
            
        self.forward_hook = {}
        for name, module in list(self.net._modules.items()):
            if isinstance(module, torch.nn.Linear):
                hook = module.register_forward_hook(self.prune_neurons)
                self.forward_hook[module] = hook
        return
        
    def remove_hook(self):       
        for module in self.forward_hook.keys():
            hook = self.forward_hook[module]
            hook.remove()
        self.forward_hook = {}
        return

In [157]:
importance = oracle_mod.compute_significance(xx, yy, mode=modes[0], normalize=True)
pmask = get_pruning_mask(importance, 1, num_prune=7)

In [158]:
importance

[tensor([2.7180e-01, 7.1302e-01, 0.0000e+00, 4.6125e-01, 5.9188e-01, 4.3188e-02,
         3.3094e-01, 2.9887e-01, 4.4542e-01, 4.7126e-03, 1.1225e+00, 1.0540e+00,
         2.5839e-01, 5.6276e-01, 8.0953e-01, 2.3213e-01, 7.8328e-01, 5.7911e-01,
         8.0413e-01, 9.7869e-02, 6.5135e-01, 4.3181e-01, 2.3574e-01, 6.9368e-02,
         7.3158e-01, 1.4478e-02, 1.7127e+00, 4.5066e-01, 4.8489e-01, 3.6393e-01,
         1.2598e+00, 5.8190e-01, 4.7238e-01, 0.0000e+00, 1.9083e-02, 4.5319e-01,
         0.0000e+00, 1.8560e-01, 0.0000e+00, 6.1826e-01, 0.0000e+00, 2.8621e-01,
         1.3466e-01, 0.0000e+00, 3.9646e-01, 5.7998e-01, 0.0000e+00, 5.0043e-01,
         6.5398e-01, 7.3030e-01, 5.8462e-01, 2.0065e+00, 4.7246e-01, 1.5259e+00,
         6.6273e-01, 0.0000e+00, 4.3994e-01, 3.7250e-01, 2.7707e-01, 1.4977e+00,
         1.6379e-01, 7.7247e-01, 0.0000e+00, 2.8800e-01, 5.6158e-01, 7.2502e-01,
         1.2063e-01, 2.6880e+00, 3.9583e-01, 4.6785e+00, 2.2986e-01, 3.4308e-01,
         3.5786e-01, 0.0000e

In [159]:
pmask

[tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

In [160]:
pnet = Pruner(net, pmask)

In [161]:
yout_normal = net.forward(xx).data.cpu()
yout_prune = pnet.forward(xx).data.cpu()

In [162]:
yout_normal.shape, yout_prune.shape

(torch.Size([60000, 10]), torch.Size([60000, 10]))

In [163]:
criterion(yout_normal, yy), criterion(yout_prune, yy)

(tensor(0.0811), tensor(0.0811))

In [164]:
((yout_prune-yout_normal)**2).sum()

tensor(0.)

## comparing for all

In [165]:
num = 200
pnet = Pruner(net)

yout_normal = net.forward(xx).data.cpu()
print("loss is ", float(criterion(yout_normal, yy)))
for i in range(4):
    print(modes[i])
    importance = oracle_mod.compute_significance(xx, yy, mode=modes[i], normalize=True)
    pmask = get_pruning_mask(importance, 1, num_prune=num)

#     for pm in pmask:
#         print(pm)
    
    yout_prune = pnet.forward(xx, prune_mask=pmask).data.cpu()
    new_err = criterion(yout_prune, yy)
    print("new error = ", float(new_err))
    
    deviation = ((yout_prune-yout_normal)**2).mean()
    print("deviation = ", float(deviation))

loss is  0.08113747835159302
oracle
new error =  2.858886241912842
deviation =  44.69500732421875
oracle_absolute
new error =  0.904782772064209
deviation =  19.14569091796875
oracle_normalized
new error =  2.2273101806640625
deviation =  42.68088912963867
oracle_abs_norm
new error =  1.6182787418365479
deviation =  20.501169204711914


In [166]:
pnet.keys

[Linear(in_features=784, out_features=256, bias=True),
 Linear(in_features=256, out_features=128, bias=True),
 Linear(in_features=128, out_features=64, bias=True),
 Linear(in_features=64, out_features=10, bias=True)]